# Setup

Install missing modules

In [ ]:
!pip install Keras-Applications
!pip install xlrd==1.2.0
!pip install loguru
!pip install torchxrayvision

Clone BrixIA from Github

In [ ]:
!git clone 'https://github.com/BrixIA/Brixia-score-COVID-19'

Include BrixIA in sys.path

In [ ]:
import sys
sys.path.append("/content/Brixia-score-COVID-19/src")

Imports

In [ ]:
from BSNet.model import BSNet
from google.colab import drive
import tensorflow as tf
import cv2
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import linear_model
import shutil
import os
from loguru import logger
from skimage.segmentation import slic
from skimage.draw import polygon_perimeter
from skimage import measure
from PIL import Image
import torch
import torchxrayvision as xrv

BASE_PATH = "/content/drive/Shareddrives/IDA covidcxr-hackaton/"

Mount GDrive

In [ ]:
drive.mount("/content/drive", force_remount=True)

# Utility

In [ ]:
def showImage(image, title):
  """
  Shows an image adding the desired title
  
  Parameters
  ----------
  image: numpy.ndarray
    The image to be shown
  title: string
    The title to be shown for the plotted image

  Returns
  -------
  None
  """
  
  if image.ndim == 2:
    plt.imshow(image, cmap = 'gray', interpolation = 'bicubic', vmin=0, vmax=255)
  else:
    plt.imshow(image, interpolation = 'bicubic', vmin=0, vmax=255)
    
  plt.title(title)
  plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
  plt.show()

In [ ]:
def readAndResize(path, size=(512,512)):
  image = cv2.imread(path)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  return cv2.resize(image, size)

In [ ]:
def imagePreprocessing(image, clip=0.01, median_filter_kernel_size=3, perc_low=2, perc_high=98):
  '''
  Given a 16 bit grayscale image, applys a series of preprocessing techniques and
  returns an 8 bit grayscale image
  '''

  assert image[0][0].dtype == np.uint16
  #Apply CLAHE
  image = cv2.createCLAHE(clipLimit=clip * 8 * 8).apply(image) #8x8 is the window size
  #Apply median blur
  image = cv2.medianBlur(image, median_filter_kernel_size)
  #Apply percentile stretching
  hist = cv2.calcHist([image], [0], None, [65536], [0, 65536])
  percent_low = image.shape[0] * image.shape[1] * perc_low / 100
  percent_high = image.shape[0] * image.shape[1] * perc_high / 100
  percentile_low = None
  percentile_high = None
  cumulative_value = 0
  for index, val in enumerate(hist):
    cumulative_value += val
    if percentile_low == None and cumulative_value > percent_low:
      percentile_low = index
    if percentile_high == None and cumulative_value > percent_high:
      percentile_high = index
      break

  alpha = (65535 / (percentile_high - percentile_low)) / 256 #normalize to 8 bit
  beta = -percentile_low * alpha
  return np.uint8(np.clip(np.float32(image) * alpha + beta, 0, 255))

Take as input a normalized image (0-1 pixel range) and returns a 0-255 image where only 0 and 255 values are allowed

In [ ]:
def denormalizeImage(image):
  return np.uint8(np.where(image<0.5, 0, 255))

Takes as input a list  of 9 images and optionally their titles and print them on a 3x3 grid

In [ ]:
def printExampleImages(images, titles=None, plots_grid = 3):
  assert plots_grid ** 2 == len(images)
  if titles != None:
    assert len(images) == len(titles)
  plt.figure(figsize=(8, 6))
  for i, image in enumerate(images):
    ax = plt.subplot(plots_grid, plots_grid, i+1)
    plt.imshow(image, cmap = 'gray', interpolation = 'bicubic', vmin=0, vmax=255)
    plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
    if titles != None:
      ax.set_title(titles[i])

# Load the model with the weights

In [ ]:
# Create the model with preloaded weights
# MAE: 0.48
model = BSNet(backbone_name='resnet18',
              input_shape=(512, 512, 1),
              input_tensor=None,
              encoder_weights=None,
              freeze_encoder=True,
              skip_connections='default',
              decoder_block_type='transpose',
              decoder_filters=(256, 128, 64, 32, 16),
              decoder_use_batchnorm=True,
              n_upsample_blocks=5,
              upsample_rates=(2, 2, 2, 2, 2),
              classes=4,
              activation='sigmoid',
              load_seg_model=True,
              seg_model_weights=BASE_PATH+'weights/segmentation-model.h5',
              freeze_segmentation=True,
              load_align_model=True,
              align_model_weights=BASE_PATH+'weights/alignment-model.h5',
              freeze_align_model=True,
              pretrain_aligment_net=False,
              explict_self_attention=True,
              load_bscore_model=True,
              bscore_model_weights=BASE_PATH+'weights/fpn_4lev_fliplr_ncl_loss03_correct_feat128-16-44.h5',
              )

Input layer shape: `[(None, 512, 512, 1)]`

# BSNet example with an image

In [ ]:
image1 = readAndResize(BASE_PATH+'TrainSet/P_1_33.png')
showImage(image1, 'Example image')

In [ ]:
in1 = np.expand_dims(image1, axis=2)
in1 = np.expand_dims(in1, axis=0)

in1 = in1/255

out1 = model[0].predict(in1)
out2 = model[1].predict(in1)
out3 = model[2].predict(in1)

In [ ]:
out3.shape

In [ ]:
showImage(np.squeeze(np.squeeze(np.round(out1*255), axis=3), axis=0), "Segmentation")

In [ ]:
out3 = np.squeeze(out3, axis = 0)

In [ ]:
argmax = np.argmax(out3, axis = 2)
argmax

In [ ]:
np.max(out3, axis = 2)

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(argmax)
# Loop over data dimensions and create text annotations.
for i in range(argmax.shape[0]):
    for j in range(argmax.shape[1]):
        ax.text(j, i, argmax[i, j], ha="center", va="center", color="w")

# Preprocess example

"Adaptive histogram equalization (CLAHE, clip:0.01), a median filtering to cope with noise (kernel size: 3), and a clipping outside the 2nd and 98th percentile"

In [ ]:
image1 = readAndResize(BASE_PATH+'TrainSet/P_1_1.png')
showImage(image1, 'Initial resized image')

In [ ]:
clahe = cv2.createCLAHE(clipLimit=0.01*8*8) #8x8 is the window size
image2 = clahe.apply(image1)
showImage(image2, "After CLAHE")

In [ ]:
image3 = cv2.medianBlur(image2, 3)
showImage(image3, "After median filtering on a 3x3 window")

In [ ]:
plt.hist(image3.ravel(),256,[0,256]); plt.show()

In [ ]:
hist = cv2.calcHist([image3],[0],None,[256],[0,256])

percent2 = 512*512*0.02
percent98 = 512*512*0.98
percentile2 = None
percentile98 = None
cumulative_value = 0

for index, val in enumerate(hist):
  cumulative_value += val
  if percentile2 == None and cumulative_value > percent2:
    percentile2 = index
  if percentile98 == None and cumulative_value > percent98:
    percentile98 = index

alpha = 255 / (percentile98 - percentile2)
beta = -255*percentile2 / (percentile98 - percentile2)

image4 = cv2.convertScaleAbs(image3, None, alpha, beta)

In [ ]:
showImage(image4, "After clipping outside the 2nd and 98th percentile")

In [ ]:
plt.hist(image4.ravel(),256,[0,256]); plt.show()

# Read Dataset

In [ ]:
df = pd.read_excel(BASE_PATH+'trainClinData.xls')
df

# Solving the issue with the images

The images that the organizers provided to us are not ready to be fed to BSNet. They are in a `.png` format (and not dicom). Hence, all the additional metadata are missing. Practically, the issue with the images are the following:

- Some of them are rotated
- Some are taken in positive and some are taken in negative

In the following we will investigate 2 tests that we can apply to the images, in particular to the segmentation mask, in order to understand if they are good, or can be adjusted or we can do nothing with the neural network.

## Lungs size

In this section we try to discover the first trivial test: the lungs size. Are the 2 biggest blobs big enough to be considered 2 lungs?

### What happens to BSNet with incorrect images?

Let's first understand what happen if you feed BSNet with an image that is not correct (e.g. is inverted or rotated).

In [ ]:
BATCH_SIZE = 9
preprocessed_images = np.empty((BATCH_SIZE, 512, 512), dtype=np.uint8)

for index, image_path in enumerate(df[df.index < BATCH_SIZE].ImageFile):
  image = readAndResize(BASE_PATH+'TrainSet/'+image_path)
  image = imagePreprocessing(image)
  preprocessed_images[index] = image

In [ ]:
printExampleImages(preprocessed_images)

In [ ]:
input_images = np.empty((BATCH_SIZE, 512, 512, 1), dtype=np.float32)
for i, image in enumerate(preprocessed_images):
  input_images[i] = np.expand_dims(image / 255, axis=2)

In [ ]:
out_images = model[0].predict(input_images)

In [ ]:
chest_masks = np.empty((BATCH_SIZE, 512, 512), dtype=np.uint8)
for i, image in enumerate(out_images):
  image = np.squeeze(image, axis=2)
  chest_masks[i] = denormalizeImage(image)

In [ ]:
printExampleImages(chest_masks)

It seems that if the image is correct we get 2 segmentated lungs (as expected). But if the images are incorrect we get noise or even nothing.

Be careful, this are just 9 images! Along all the thousand images there are more strange cases.

### Erosion of the masks



Let's apply a little bit of erosion. The lungs stays lungs and we get rid of a little bit of noise.

In [ ]:
eroded_images = np.empty((BATCH_SIZE, 512, 512), dtype=np.uint8)
for i, image in enumerate(chest_masks):
  eroded_images[i] = cv2.erode(image, np.ones((3,3), np.uint8), iterations=3)

In [ ]:
printExampleImages(eroded_images)

### Count the number of connected components

If we identify 2 regions with at least `num_pixels_lung` pixels (we have to exclude the background region) then the image is ok

In [ ]:
num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(eroded_images[0], 8, cv2.CV_32S)

In [ ]:
stats

In this vector the last column represent the number of pixel of the respective region.

Be careful! The `stats` matrix is not sorted by the number of pixels!

In [ ]:
plt.imshow(labels, cmap = 'gray', interpolation = 'bicubic', vmin=0, vmax=num_labels-1)

### Adjust positive/negative images

Here we try to combine everything we said unitl now in a function. If the image pass the test than the same image is returned, otherwise we invert it. Let's see the results.

In [ ]:
def adjust_positive_negative(image, num_pixels_lung=10000):
  # Get the preprocessed image
  initial_image = image
  # Adapt to the dimensions requested by BSNet
  image = np.expand_dims(np.expand_dims(initial_image / 255, axis=2), axis=0)
  # Produce the binary mask
  image = model[0].predict(image)
  # Go back to 512x512 image format
  image = np.squeeze(np.squeeze(image, axis=0), axis=2)
  # Use the 0-255 pixel value range
  image = denormalizeImage(image)
  # Erode the mask
  image = cv2.erode(image, np.ones((3,3), np.uint8), iterations=3)
  # Find connected components
  _, _, stats, _ = cv2.connectedComponentsWithStats(image, 8, cv2.CV_32S)
  # Sort stats by the last column
  argsort = np.argsort(stats[:, -1])[::-1]
  stats = stats[argsort]
  # If the first and the second region has at least 10000 pixel then it's ok
  if(len(stats)>=3 and stats[1][-1] > num_pixels_lung and stats[2][-1] > num_pixels_lung):
    return initial_image, False
  return 255 - initial_image, True

In [ ]:
printExampleImages(preprocessed_images)

In [ ]:
adjusted_images = np.empty_like(preprocessed_images)
titles = []
for i, image in enumerate(preprocessed_images):
  adjusted_images[i], adjusted = adjust_positive_negative(image)
  titles.append("Adjusted" if adjusted else "Already OK")

printExampleImages(adjusted_images, titles)

It seems that we are on the rigth path. But this is only a trivial test. We have no guarantees that 2 big blobs are 2 lungs, could be noise.

## Lungs segmentation assessment


In this section we introduce another test. Are the 2 big blobs something similar to 2 lungs? We can try to use some information produced by `cv2.connectedComponentsWithStats()` in order to fit 2 lines on the 2 lungs and try to underestand if the lines converges in a point in a certain location.

In [ ]:
input_images = np.empty((BATCH_SIZE, 512, 512, 1), dtype=np.float32)
for i, image in enumerate(adjusted_images):
  input_images[i] = np.expand_dims(image / 255, axis=2)

In [ ]:
out_images = model[0].predict(input_images)

In [ ]:
chest_masks = np.empty((BATCH_SIZE, 512, 512), dtype=np.uint8)
for i, image in enumerate(out_images):
  image = np.squeeze(image, axis=2)
  chest_masks[i] = denormalizeImage(image)

In [ ]:
eroded_images = np.empty((BATCH_SIZE, 512, 512), dtype=np.uint8)
for i, image in enumerate(chest_masks):
  eroded_images[i] = cv2.erode(image, np.ones((3,3), np.uint8), iterations=3)

In [ ]:
printExampleImages(eroded_images)

In [ ]:
showImage(eroded_images[0], "We will work on this image")

In [ ]:
num_regions, labels, _, _ = cv2.connectedComponentsWithStats(eroded_images[0], 8, cv2.CV_32S)

In [ ]:
lung_1 = np.argwhere(labels==1)
y_lung_1 = lung_1[:,0]
x_lung_1 = lung_1[:,1]
lung_2 = np.argwhere(labels==2)
y_lung_2 = lung_2[:,0]
x_lung_2 = lung_2[:,1]

In the following we will try some techniques for line fitting.

### Least square

In [ ]:
L_1 = np.polyfit(x_lung_1, -y_lung_1, 1)
L_2 = np.polyfit(x_lung_2, -y_lung_2, 1)
L_1, L_2

In [ ]:
x_intersection = (L_2[1]-L_1[1])/(L_1[0]-L_2[0])
y_intersection = L_1[0]*x_intersection + L_1[1]
intersection = np.array([x_intersection, y_intersection])

In [ ]:
xp = np.linspace(0, 512, 513)
p_1 = np.poly1d(L_1)
p_2 = np.poly1d(L_2)
plt.plot(x_lung_1, -y_lung_1, '.', xp, p_1(xp), '-', x_lung_2, -y_lung_2, '.', xp, p_2(xp), '-', intersection[0], intersection[1], 'o')
plt.xlim([0, 512])
plt.ylim([-512, 0])
plt.axes().set_aspect('equal')

Despite this image is very well segmentated, the line direction of the left lung is not perfect.

### RANSAC

In [ ]:
ransac_1 = linear_model.RANSACRegressor(linear_model.LinearRegression())
ransac_1.fit(x_lung_1.reshape(-1,1), -y_lung_1)

ransac_2 = linear_model.RANSACRegressor(linear_model.LinearRegression())
ransac_2.fit(x_lung_2.reshape(-1,1), -y_lung_2)

In [ ]:
R_1 = np.array([ransac_1.estimator_.coef_[0], ransac_1.predict([[0]])[0]])
R_2 = np.array([ransac_2.estimator_.coef_[0], ransac_2.predict([[0]])[0]])
R_1, R_2

In [ ]:
x_intersection = (R_2[1]-R_1[1])/(R_1[0]-R_2[0])
y_intersection = R_1[0]*x_intersection + R_1[1]
intersection = np.array([x_intersection, y_intersection])

In [ ]:
xp = np.linspace(0, 512, 513)
p_1 = np.poly1d(R_1)
p_2 = np.poly1d(R_2)
plt.plot(x_lung_1, -y_lung_1, '.', xp, p_1(xp), '-', x_lung_2, -y_lung_2, '.', xp, p_2(xp), '-', intersection[0], intersection[1], 'o')
plt.xlim([0, 512])
plt.ylim([-512, 0])
plt.axes().set_aspect('equal')

With RANSAC we get a quite perfect line fitting on the 2 lungs. But let's try yet another technique.

### Ellipse fitting

In this technique we try to perform an ellipse fitting on each of the two lungs. Then, we compute the line as the direction of the axe of the ellipse. Then we perform the same operations that we did before.

In [ ]:
num_regions, labels, stats, _ = cv2.connectedComponentsWithStats(eroded_images[0], 8, cv2.CV_32S)
# Sort stats by the last column
argsort = np.argsort(stats[:, -1])[::-1]
stats = stats[argsort]

In [ ]:
lungs = np.uint8(np.where(np.logical_or(np.equal(labels, argsort[1]), np.equal(labels, argsort[2])), 255, 0))
showImage(lungs, "Lungs")

In [ ]:
canny_output = cv2.Canny(lungs, 100, 200)
contours, _ = cv2.findContours(canny_output, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# Keep just the 2 longest contours
argsort_contour = np.argsort([len(val) for val in contours])[-2:]
# Redefine contours with just the 2 longest contours
contours = [contours[i] for i in argsort_contour]
# Find the ellipses for each contour
minEllipses = [None]*len(contours)
for i, c in enumerate(contours):
  if c.shape[0] > 5:
    minEllipses[i] = cv2.fitEllipse(c)

# Draw contours + rotated rects + ellipses
    
drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)
    
for i, c in enumerate(contours):
  color = 255, 255, 255
  # contour
  cv2.drawContours(drawing, contours, i, color)
  # ellipse
  if c.shape[0] > 5:
    cv2.ellipse(drawing, minEllipses[i], color, 2)
showImage(drawing, "Fitted ellipse")

In [ ]:
minEllipses

This is 
- center	The rectangle mass center.
- size	Width and height of the rectangle.
- angle	The rotation angle in a clockwise direction. When the angle is 0, 90, 180, 270 etc., the rectangle becomes an up-right rectangle.


In [ ]:
slope_1 = np.tan(np.deg2rad(90 - minEllipses[0][2]))
slope_2 = np.tan(np.deg2rad(90 - minEllipses[1][2]))
intercept_1 = -minEllipses[0][0][1] - slope_1 * minEllipses[0][0][0]
intercept_2 = -minEllipses[1][0][1] - slope_2 * minEllipses[1][0][0]

x_intersection = (intercept_2 - intercept_1) / (slope_1 - slope_2)
y_intersection = slope_1 * x_intersection + intercept_1

In [ ]:
lungs_points = np.argwhere(np.logical_or(np.equal(labels, argsort[1]), np.equal(labels, argsort[2])))
y_lungs = lungs_points[:,0]
x_lungs = lungs_points[:,1]

In [ ]:
xc = np.arange(0, 512)
p_1 = np.poly1d([slope_1, intercept_1])
p_2 = np.poly1d([slope_2, intercept_2])

plt.plot(x_lungs, -y_lungs, '.', xc, p_1(xc), '-', xc, p_2(xc), '-', x_intersection, y_intersection, 'o')
plt.xlim([0, 512])
plt.ylim([-512, 200])
plt.axes().set_aspect('equal')

Well it seems that the result is even more accurate than RANSAC

## Definition of a compact function

Here we define a function that did everything we did in the last 3 subsections. You can specify the technique you want and you can even submit precomputed `connectedComponents` (we will see why this is useful in the following sections.

In [ ]:
def assess_lungs_direction(image, connectedComponents=None, assessment_mode="ellipse_fitting") -> bool:
  if connectedComponents is None:
    num_regions, labels, stats, _ = cv2.connectedComponentsWithStats(image, 8, cv2.CV_32S)
  else:
    num_regions, labels, stats, _ = connectedComponents
  
  if num_regions < 3:
    return False
  argsort = np.argsort(stats[:, -1])[::-1]

  if assessment_mode == "least_square":
    lung_1 = np.argwhere(labels==argsort[1])
    y_lung_1 = lung_1[:,0]
    x_lung_1 = lung_1[:,1]
    lung_2 = np.argwhere(labels==argsort[2])
    y_lung_2 = lung_2[:,0]
    x_lung_2 = lung_2[:,1]
    slope_1, intercept_1 = np.polyfit(x_lung_1, -y_lung_1, 1)
    slope_2, intercept_2 = np.polyfit(x_lung_2, -y_lung_2, 1)

  elif assessment_mode == "ransac":
    lung_1 = np.argwhere(labels==argsort[1])
    y_lung_1 = lung_1[:,0]
    x_lung_1 = lung_1[:,1]
    lung_2 = np.argwhere(labels==argsort[2])
    y_lung_2 = lung_2[:,0]
    x_lung_2 = lung_2[:,1]
    ransac_1 = linear_model.RANSACRegressor(linear_model.LinearRegression())
    ransac_1.fit(x_lung_1.reshape(-1,1), -y_lung_1)
    ransac_2 = linear_model.RANSACRegressor(linear_model.LinearRegression())
    ransac_2.fit(x_lung_2.reshape(-1,1), -y_lung_2)
    slope_1 = ransac_1.estimator_.coef_[0]
    slope_2 = ransac_2.estimator_.coef_[0]
    intercept_1 = ransac_1.predict([[0]])[0]
    intercept_2 = ransac_2.predict([[0]])[0]

  elif assessment_mode == "ellipse_fitting":
    lungs = np.uint8(np.where(np.logical_or(np.equal(labels, argsort[1]), np.equal(labels, argsort[2])), 255, 0))
    canny_output = cv2.Canny(lungs, 100, 200)
    contours, _ = cv2.findContours(canny_output, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Keep just the 2 longest contours
    argsort_contour = np.argsort([len(val) for val in contours])[-2:]
    # Redefine contours with just the 2 longest contours
    contours = [contours[i] for i in argsort_contour]
    # Find the ellipses for each contour
    minEllipses = [None]*len(contours)
    for i, c in enumerate(contours):
      if c.shape[0] > 5:
        minEllipses[i] = cv2.fitEllipse(c)
    slope_1 = np.tan(np.deg2rad(90 - minEllipses[0][2]))
    slope_2 = np.tan(np.deg2rad(90 - minEllipses[1][2]))
    intercept_1 = -minEllipses[0][0][1] - slope_1 * minEllipses[0][0][0]
    intercept_2 = -minEllipses[1][0][1] - slope_2 * minEllipses[1][0][0]

  else:
    raise ValueError('assessment_mode must be either least_square, ransac or ellipse_fitting')

  x_intersection = (intercept_2 - intercept_1) / (slope_1 - slope_2)
  y_intersection = slope_1 * x_intersection + intercept_1

  if y_intersection > -256 and x_intersection > 0 and x_intersection < 512:
    return True
  return False

In [ ]:
%timeit ["OK" if assess_lungs_direction(image, "least_square") else "NOT OK" for image in eroded_images]

10 loops, best of 5: 63.2 ms per loop


In [ ]:
%timeit ["OK" if assess_lungs_direction(image, "ransac") else "NOT OK" for image in eroded_images]

10 loops, best of 5: 195 ms per loop


In [ ]:
%timeit ["OK" if assess_lungs_direction(image, "ellipse_fitting") else "NOT OK" for image in eroded_images]

10 loops, best of 5: 28.1 ms per loop


In [ ]:
titles = ["OK" if assess_lungs_direction(image, "ellipse_fitting") else "NOT OK" for image in eroded_images]
printExampleImages(eroded_images, titles)

Guess what, ellipse fitting is the fastest tecnique (and even by a big margin). Plus we can say that after many tests it is sligthy more accurate than RANSAC.

# Adjust the images on TrainSet

At the moment we introduced 2 tests in order to understand if the segmentated images are good. Now let's try to adjust the images with the help of the 2 previous tests.

How many possibile correction can we apply to the images? 4 possibile rotation and each of them could be inverted or not. So they are just 8 possibile combination. And if we take a look to the images most of them are no rotated, so we have just to invert them. In this case a bruteforce approach is one of the easier thing we can do. What does it mean?


1.   The input image pass all the tests? Return it
2.   Otherwise **invert** it. Now it pass all the tests? Return it
3.   Otherwise **rotate** it. Now it pass all the tests? Return it
4.   Otherwise **invert** it. Now it pass all the tests? Return it
5.   Otherwise **rotate** it. Now it pass all the tests? Return it
6.   Otherwise **invert** it. Now it pass all the tests? Return it
7.   Otherwise **rotate** it. Now it pass all the tests? Return it
8.   Otherwise **invert** it. Now it pass all the tests? Return it
9.   Otherwise it is **unrecoverable**.



In [ ]:
#Brute-force lungs segmentation assessment
def adjust_preprocessed_image(image, num_pixels_lung=10000):
  # Adapt to the dimensions requested by BSNet
  adapted_image = np.expand_dims(np.expand_dims(image / 255, axis=2), axis=0)
  # most images are not rotated (0), few of them are rotate buy just 90 degrees (3, 1),
  # almost none of them are rotated by 180 degree (2)
  for rotation_count in [0, 3, 1, 2]:
    rotated_image = np.rot90(adapted_image, k=rotation_count, axes=(1,2))
    print("Rotation count ", rotation_count)
    for pos_neg in [False, True]:
      if pos_neg:
        print("Inverting...")
        rotated_image = 1 - rotated_image
      print("Computing the segmentation...")
      mask = model[0].predict(rotated_image)
      print("Components size assessment...")
      # Go back to 512x512 image format
      mask = np.squeeze(np.squeeze(mask, axis=0), axis=2)
      # Use the 0-255 pixel value range
      mask = denormalizeImage(mask)
      # Erode the mask
      mask = cv2.erode(mask, np.ones((3,3), np.uint8), iterations=3)
      # Find connected components
      connectedComponents = cv2.connectedComponentsWithStats(mask, 8, cv2.CV_32S)
      num_regions, _, stats, _ = connectedComponents
      # Sort stats by the last column
      argsort = np.argsort(stats[:, -1])[::-1]
      stats = stats[argsort]
      # If the first and the second region has at least 10000 pixel then it's ok
      if num_regions>=3 and stats[1][-1] > num_pixels_lung and stats[2][-1] > num_pixels_lung:
        print("Components size OK\n Computing lungs direction")
        if(assess_lungs_direction(mask, connectedComponents=connectedComponents, assessment_mode="ellipse_fitting")):
          print("Lungs direction OK")
          return np.squeeze(np.squeeze(np.uint8(rotated_image * 255), axis = 0), axis = 2)
  return None

## Example with one image

In [ ]:
exampleImage = cv2.imread(BASE_PATH+"TrainSetPreprocessed/P_1_60.png", 0)
showImage(exampleImage, "Preprocessed example image")

This is an example of rotated and inverted image, let's see if the algorithm adjust it.

In [ ]:
showImage(adjust_preprocessed_image(exampleImage), "After correction")

## Apply to all the Training set and put adjusted images in a folder

This function is slighty different then the one above

In [ ]:
#Brute-force lungs segmentation assessment
def adjust_preprocessed_image(image, num_pixels_lung=10000):
  # Adapt to the dimensions requested by BSNet
  adapted_image = np.expand_dims(np.expand_dims(image / 255, axis=2), axis=0)
  # most images are not rotated (0), few of them are rotate buy just 90 degrees (3, 1),
  # almost none of them are rotated by 180 degree (2)
  for rotation_count in [0, 3, 1, 2]:
    rotated_image = np.rot90(adapted_image, k=rotation_count, axes=(1,2))
    for pos_neg in [False, True]:
      if pos_neg:
        rotated_image = 1 - rotated_image
      mask = model[0].predict(rotated_image)
      # Go back to 512x512 image format
      mask = np.squeeze(np.squeeze(mask, axis=0), axis=2)
      # Use the 0-255 pixel value range
      mask = denormalizeImage(mask)
      # Erode the mask
      mask = cv2.erode(mask, np.ones((3,3), np.uint8), iterations=3)
      # Find connected components
      connectedComponents = cv2.connectedComponentsWithStats(mask, 8, cv2.CV_32S)
      num_regions, _, stats, _ = connectedComponents
      # Sort stats by the last column
      argsort = np.argsort(stats[:, -1])[::-1]
      stats = stats[argsort]
      # If the first and the second region has at least 10000 pixel then it's ok
      if num_regions>=3 and stats[1][-1] > num_pixels_lung and stats[2][-1] > num_pixels_lung:
        if assess_lungs_direction(mask, connectedComponents=connectedComponents, assessment_mode="ellipse_fitting"):
          return np.squeeze(np.squeeze(np.uint8(rotated_image * 255), axis = 0), axis = 2)
  return None

In [ ]:
file_names = os.listdir(BASE_PATH+"TrainSetPreprocessed/")

for index, file_name in enumerate(file_names):
  print("\rProcessing: "+str(index+1)+"/"+str(len(file_names)), end="")
  image = cv2.imread(BASE_PATH+'TrainSetPreprocessed/'+file_name, cv2.IMREAD_GRAYSCALE)
  output_image = adjust_preprocessed_image(image)
  if output_image is None:
    shutil.copyfile(BASE_PATH+'TrainSetPreprocessed/'+file_name, BASE_PATH+'TrainSetNotAdjusted/'+file_name)
  else:
    cv2.imwrite(BASE_PATH+'TrainSetAdjusted/'+file_name, output_image)

Show all the images that has been discarded

In [ ]:
file_names = os.listdir(BASE_PATH+"TrainSetNotAdjusted/")

for index, file_name in enumerate(file_names):
  #print("\rProcessing: "+str(index+1)+"/"+str(len(file_names)), end="")
  image = cv2.imread(BASE_PATH+'TrainSetPreprocessed/'+file_name, cv2.IMREAD_GRAYSCALE)
  print(file_name)
  showImage(image, file_name)

In [ ]:
file_names = os.listdir(BASE_PATH+"TrainSetNotAdjusted/")

images_to_discard = [
                     "P_296.png",
                     "P_612.png",
                     "P_423.png",
                     "P_311.png",
                     "P_159.png",
                     "P_1_137.png",
                     "P_270.png",
                     "P_2_98.png",
                     "P_141.png",
                     "P_1_34.png",
                     "P_476.png",
                     "P_2_58.png",
                     "P_2_116.png",
]

images_to_keep = np.setdiff1d(np.array(file_names), np.array(images_to_discard)).tolist()

Move `images_to_keep` in TrainSetAdjusted

In [ ]:
for file_name in images_to_keep:
  shutil.move(BASE_PATH+'TrainSetNotAdjusted/'+file_name, BASE_PATH+'TrainSetAdjusted/'+file_name)

After manual inspection some images has been moved to `TrainSetNotAdjusted` and some other need to be inverted

In [ ]:
images_to_invert = [
                    "P_1_11.png",
                    "P_1_13.png",
                    "P_1_14.png",
                    "P_1_41.png",
                    "P_1_45.png",
                    "P_1_50.png",
                    "P_1_72.png",
                    "P_1_114.png",
                    "P_1_117.png",
                    "P_1_129.png",
                    "P_4.png",
                    "P_9.png",
                    "P_54.png",
                    "P_82.png",
                    "P_119.png",
                    "P_128.png",
                    "P_177.png",
                    "P_209.png",
                    "P_210.png",
                    "P_253.png",
                    "P_256.png",
                    "P_268.png",
                    "P_279.png",
                    "P_297.png",
                    "P_360.png",
                    "P_371.png",
                    "P_396.png",
                    "P_429.png",
                    "P_436.png",
                    "P_464.png",
                    "P_468.png",
                    "P_475.png",
                    "P_523.png",
                    "P_526.png",
                    "P_664.png",
                    "P_686.png",
                    "P_689.png",
                    "P_695.png",
                    "P_700.png",
                    "P_712.png",
                    "P_729.png",
                    "P_734.png",
                    "P_753.png",
                    "P_758.png",
                    "P_760.png",
                    "P_787.png",
                    "P_788.png",
                    "P_796.png",
                    "P_802.png",
                    "P_812.png",
                    "P_815.png",
                    "P_823.png",
                    "P_827.png",
                    "P_837.png",
                    "P_840.png",
                    "P_841.png",
]

# They seems a lot, but in this list there are a lot of images that has been
# discarded from the algorithm

#P_2_9, P_2_84, P_2_121, P_138, P_382, P_451, P_747, P_803 da girare, --> DONE
#P308 da girare e invertire  --> DONE


Here we invert images that should be inverted

In [ ]:
for index, file_name in enumerate(images_to_invert):
  print(file_name)
  image = cv2.imread(BASE_PATH+'TrainSetAdjusted/'+file_name, cv2.IMREAD_GRAYSCALE)
  image = 255 - image
  cv2.imwrite(BASE_PATH+'TrainSetAdjusted/'+file_name, image)

After manual inspection some images need to be discarded

In [ ]:
images_to_discard = [
                          "P_2_11.png",
                          "P_110.png",
                          "P_269.png",
                          "P_276.png",
                          "P_445.png",
                          "P_737.png",
]

for file_name in images_to_discard:
  shutil.move(BASE_PATH+'TrainSetAdjusted/'+file_name, BASE_PATH+'TrainSetNotAdjusted/'+file_name)

# Adjust the images on TestSet

Slighty different then the one above

In [ ]:
#Brute-force lungs segmentation assessment
def adjust_preprocessed_image(image, num_pixels_lung=10000):
  # Adapt to the dimensions requested by BSNet
  adapted_image = np.expand_dims(np.expand_dims(image / 255, axis=2), axis=0)
  # most images are not rotated (0), few of them are rotate buy just 90 degrees (3, 1),
  # almost none of them are rotated by 180 degree (2)
  for rotation_count in [0]:
    rotated_image = np.rot90(adapted_image, k=rotation_count, axes=(1,2))
    for pos_neg in [False, True]:
      if pos_neg:
        rotated_image = 1 - rotated_image
      mask = model[0].predict(rotated_image)
      # Go back to 512x512 image format
      mask = np.squeeze(np.squeeze(mask, axis=0), axis=2)
      # Use the 0-255 pixel value range
      mask = denormalizeImage(mask)
      # Erode the mask
      mask = cv2.erode(mask, np.ones((3,3), np.uint8), iterations=3)
      # Find connected components
      connectedComponents = cv2.connectedComponentsWithStats(mask, 8, cv2.CV_32S)
      num_regions, _, stats, _ = connectedComponents
      # Sort stats by the last column
      argsort = np.argsort(stats[:, -1])[::-1]
      stats = stats[argsort]
      # If the first and the second region has at least 10000 pixel then it's ok
      if num_regions>=3 and stats[1][-1] > num_pixels_lung and stats[2][-1] > num_pixels_lung:
        if assess_lungs_direction(mask, connectedComponents=connectedComponents, assessment_mode="ellipse_fitting"):
          return np.squeeze(np.squeeze(np.uint8(rotated_image * 255), axis = 0), axis = 2)
  return None

In [ ]:
file_names = os.listdir(BASE_PATH+"TestSetPreprocessed/")

for index, file_name in enumerate(file_names):
  print("\rProcessing: "+str(index+1)+"/"+str(len(file_names)), end="")
  image = cv2.imread(BASE_PATH+'TestSetPreprocessed/'+file_name, cv2.IMREAD_GRAYSCALE)
  output_image = adjust_preprocessed_image(image)
  if output_image is None:
    shutil.copyfile(BASE_PATH+'TestSetPreprocessed/'+file_name, BASE_PATH+'TestSetNotAdjusted/'+file_name)
  else:
    cv2.imwrite(BASE_PATH+'TestSetAdjusted/'+file_name, output_image)

Show all the images that has been discarded

In [ ]:
file_names = os.listdir(BASE_PATH+"TestSetNotAdjusted/")

for index, file_name in enumerate(file_names):
  #print("\rProcessing: "+str(index+1)+"/"+str(len(file_names)), end="")
  image = cv2.imread(BASE_PATH+'TestSetPreprocessed/'+file_name, cv2.IMREAD_GRAYSCALE)
  print(file_name)
  showImage(image, file_name)

In [ ]:
file_names = os.listdir(BASE_PATH+"TestSetNotAdjusted/")

images_to_discard = [
                     "P_3_335.png",
]

images_to_keep = np.setdiff1d(np.array(file_names), np.array(images_to_discard)).tolist()

Move `images_to_keep` in TrainSetAdjusted

In [ ]:
for file_name in images_to_keep:
  image = cv2.imread(BASE_PATH + "TestSetNotAdjusted/" + file_name, cv2.IMREAD_GRAYSCALE)
  image = 255 - image   #invert before moving
  cv2.imwrite(BASE_PATH + "TestSetAdjusted/" + file_name, image)
  os.remove(BASE_PATH + "TestSetNotAdjusted/" + file_name)

After manual inspection some images has been moved to `TestSetNotAdjusted` and some other need to be inverted

In [ ]:
images_to_invert = [
                    "P_3_99.png",
                    "P_3_184.png",
                    "P_3_213.png",
                    "P_3_336.png",
]


Here we invert images that should be inverted

In [ ]:
for index, file_name in enumerate(images_to_invert):
  print(file_name)
  image = cv2.imread(BASE_PATH+'TestSetAdjusted/'+file_name, cv2.IMREAD_GRAYSCALE)
  image = 255 - image
  cv2.imwrite(BASE_PATH+'TestSetAdjusted/'+file_name, image)

# Compute the BS score on the Training set

Now let's compute the BS-score on the adjusted images

In [ ]:
DF_SIZE = len(df)

correctness = np.zeros((DF_SIZE), dtype=np.uint8)
bs_score = np.empty((DF_SIZE, 3, 2, 4), dtype=np.float32)

for index, image_path in enumerate(df.ImageFile):
  print("\rProcessing: "+str(index+1)+"/"+str(DF_SIZE), end="")

  file_path = BASE_PATH+'TrainSetAdjusted/'+image_path
  # if the image exists in that folder then it is ok, otherwise it means that
  # it has been discarded in the previous stage
  if os.path.exists(file_path):
    image = np.expand_dims(np.expand_dims(cv2.imread(file_path, cv2.IMREAD_GRAYSCALE), axis = 0), axis = 3) / 255
    correctness[index] = 1
    bs_score[index] = np.squeeze(model[-1].predict(image), axis = 0)
  else:
    correctness[index] = 0
    bs_score[index].fill(0.25)

In [ ]:
print("Amount of bad images: "+str(DF_SIZE - np.count_nonzero(correctness))+"/"+str(DF_SIZE))

In [ ]:
argmax = np.argmax(bs_score, axis = 3)
argmax = np.reshape(argmax, (DF_SIZE, 6))
probabilities = np.max(bs_score, axis = 3)
probabilities = np.reshape(probabilities, (DF_SIZE, 6))
confidence = np.mean(probabilities, axis=1)

In [ ]:
for i in range(6):
  df["Brixia_score_"+str(i)] = argmax[:,i]

for i in range(6):
  df["Brixia_score_prob_"+str(i)] = probabilities[:,i]

df["Brixia_score_correctness"] = correctness
df["Brixia_score_confidence"] = confidence
df

In [ ]:
df.Brixia_score_confidence.mean()

In [ ]:
df.to_excel(BASE_PATH+"trainClinData_brixiascore.xls")

# Compute the BS score on the Test set

Now let's compute the BS-score on the adjusted images

In [ ]:
df = pd.read_excel(BASE_PATH+"testClinData.xls")

DF_SIZE = len(df)

correctness = np.zeros((DF_SIZE), dtype=np.uint8)
bs_score = np.empty((DF_SIZE, 3, 2, 4), dtype=np.float32)

for index, image_path in enumerate(df.ImageFile):
  print("\rProcessing: "+str(index+1)+"/"+str(DF_SIZE), end="")

  file_path = BASE_PATH+'TestSetAdjusted/'+image_path
  # if the image exists in that folder then it is ok, otherwise it means that
  # it has been discarded in the previous stage
  if os.path.exists(file_path):
    image = np.expand_dims(np.expand_dims(cv2.imread(file_path, cv2.IMREAD_GRAYSCALE), axis = 0), axis = 3) / 255
    correctness[index] = 1
    bs_score[index] = np.squeeze(model[-1].predict(image), axis = 0)
  else:
    correctness[index] = 0
    bs_score[index].fill(0.25)

In [ ]:
print("Amount of bad images: "+str(DF_SIZE - np.count_nonzero(correctness))+"/"+str(DF_SIZE))

In [ ]:
argmax = np.argmax(bs_score, axis = 3)
argmax = np.reshape(argmax, (DF_SIZE, 6))
probabilities = np.max(bs_score, axis = 3)
probabilities = np.reshape(probabilities, (DF_SIZE, 6))
confidence = np.mean(probabilities, axis=1)

In [ ]:
for i in range(6):
  df["Brixia_score_"+str(i)] = argmax[:,i]

for i in range(6):
  df["Brixia_score_prob_"+str(i)] = probabilities[:,i]

df["Brixia_score_correctness"] = correctness
df["Brixia_score_confidence"] = confidence
df

In [ ]:
df.Brixia_score_confidence.mean()

In [ ]:
df.to_excel(BASE_PATH+"testClinData_brixiascore.xls")

## Load the model

In [ ]:
heart_model = xrv.models.ResNet(weights="resnet50-res512-all")
heart_model.op_threshs = None # prevent pre-trained model calibration
heart_model.model.fc = torch.nn.Linear(2048,1) # reinitialize classifier
heart_model.load_state_dict(torch.load(BASE_PATH + 'best_metric_model_so_far.pth'))
heart_model=heart_model.to("cuda")
heart_model.eval()

In [ ]:
# Normalize image from -1024 to 1024
def normalize(img, maxval=255, reshape=False):
  """Scales images to be roughly [-1024 1024]."""

  if img.max() > maxval:
    raise Exception("max image value ({}) higher than expected bound ({}).".format(img.max(), maxval))

  img = (2 * (img.astype(np.float32) / maxval) - 1.) * 1024

  if reshape:
    # Check that images are 2D arrays
    if len(img.shape) > 2:
      img = img[:, :, 0]
    if len(img.shape) < 2:
      print("error, dimension lower than 2 for image")

    # add color channel
    img = img[None, :, :]

  return img

## Example with one image

In [ ]:
#image = cv2.imread(BASE_PATH+"TrainSetAdjusted/P_703.png", cv2.IMREAD_GRAYSCALE)
image = cv2.imread(BASE_PATH+"TrainSetAdjusted/P_2_108.png", cv2.IMREAD_GRAYSCALE)
image = np.expand_dims(np.expand_dims(normalize(image) , axis=0), axis=0)

In [ ]:
image.shape

In [ ]:
np.min(image)

In [ ]:
heart_model(torch.from_numpy(image).float().to("cuda"))#.cpu().data.numpy()[0][0]

## Compute on all the images

In [ ]:
df = pd.read_excel(BASE_PATH + "trainClinData_brixiascore.xls")

DF_SIZE = len(df)

heart_score = np.empty((DF_SIZE), dtype=np.float32)

for index, image_path in enumerate(df.ImageFile):
  print("\rProcessing: "+str(index+1)+"/"+str(DF_SIZE), end="")

  file_path = BASE_PATH+'TrainSetAdjusted/'+image_path
  # if the image exists in that folder then it is ok, otherwise it means that
  # it has been discarded in the previous stage
  if os.path.exists(file_path):
    image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    image = np.expand_dims(np.expand_dims(normalize(image), axis = 0), axis = 0)
    image = torch.from_numpy(image).float().to("cuda")
    heart_score[index] = heart_model(image).cpu().data.numpy()[0][0]
  else:
    heart_score[index] = 0.5

In [ ]:
df["heart_score"] = heart_score
df

In [ ]:
np.max(heart_score)

In [ ]:
np.min(heart_score)

In [ ]:
df.to_excel(BASE_PATH+"trainClinData_brixiascore_heartscore.xls")

# Process Test Set and compute the BS score

Here we do the same thing we did with the training set but here we do it for the test set

In [ ]:
df_test = pd.read_excel(BASE_PATH+'testClinData.xls')
df_test

In [ ]:
#Brute-force lungs segmentation assessment
def adjust_preprocessed_image(image, num_pixels_lung=10000):
  # Adapt to the dimensions requested by BSNet
  adapted_image = np.expand_dims(np.expand_dims(image / 255, axis=2), axis=0)
  # most images are not rotated (0), few of them are rotate buy just 90 degrees (3, 1),
  # almost none of them are rotated by 180 degree (2)
  for rotation_count in [0, 3, 1, 2]:
    rotated_image = np.rot90(adapted_image, k=rotation_count, axes=(1,2))
    for pos_neg in [False, True]:
      if pos_neg:
        rotated_image = 1 - rotated_image
      mask = model[0].predict(rotated_image)
      # Go back to 512x512 image format
      mask = np.squeeze(np.squeeze(mask, axis=0), axis=2)
      # Use the 0-255 pixel value range
      mask = denormalizeImage(mask)
      # Erode the mask
      mask = cv2.erode(mask, np.ones((3,3), np.uint8), iterations=3)
      # Find connected components
      connectedComponents = cv2.connectedComponentsWithStats(mask, 8, cv2.CV_32S)
      num_regions, _, stats, _ = connectedComponents
      # Sort stats by the last column
      argsort = np.argsort(stats[:, -1])[::-1]
      stats = stats[argsort]
      # If the first and the second region has at least 10000 pixel then it's ok
      if num_regions>=3 and stats[1][-1] > num_pixels_lung and stats[2][-1] > num_pixels_lung:
        if(assess_lungs_direction(mask, connectedComponents=connectedComponents, assessment_mode="ellipse_fitting")):
          return rotated_image
  return None

In [ ]:
DF_SIZE = len(df_test)

correctness = np.zeros((DF_SIZE), dtype=np.uint8)
bs_score = np.empty((DF_SIZE, 3, 2, 4), dtype=np.float32)

for index, image_path in enumerate(df_test.ImageFile):
  print("\rProcessing: "+str(index+1)+"/"+str(DF_SIZE), end="")
  image = cv2.imread(BASE_PATH+'TestSetPreprocessed/'+image_path, cv2.IMREAD_GRAYSCALE)
  output_image = adjust_preprocessed_image(image)
  if output_image is None:
    correctness[index] = 0
    bs_score[index].fill(0.25)
  else:
    correctness[index] = 1
    bs_score[index] = np.squeeze(model[-1].predict(output_image), axis = 0)

In [ ]:
print("Amount of bad images: "+str(DF_SIZE - np.count_nonzero(correctness))+"/"+str(DF_SIZE))

In [ ]:
argmax = np.argmax(bs_score, axis = 3)
argmax = np.reshape(argmax, (DF_SIZE, 6))
probabilities = np.max(bs_score, axis = 3)
probabilities = np.reshape(probabilities, (DF_SIZE, 6))
confidence = np.mean(probabilities, axis=1)

In [ ]:
for i in range(6):
  df_test["Brixia_score_"+str(i)] = argmax[:,i]

for i in range(6):
  df_test["Brixia_score_prob_"+str(i)] = probabilities[:,i]

df_test["Brixia_score_correctness"] = correctness
df_test["Brixia_score_confidence"] = confidence
df_test

In [ ]:
df_test.to_excel(BASE_PATH+"testClinData_brixiascore.xls")

## Visualize bad images

In [ ]:
df_test_bad_images = df_test[df_test.Brixia_score_correctness == 0]
df_test_bad_images

In [ ]:
# Little bit different function then before. Doesn't rotate and always returns something
def get_max_non_zero_image(image):
  # Adapt to the dimensions requested by BSNet
  adapted_image = np.expand_dims(np.expand_dims(image / 255, axis=2), axis=0)
  # most images are not rotated (0), few of them are rotate buy just 90 degrees (3, 1),
  # almost none of them are rotated by 180 degree (2)
  max_non_zero_pixels = 0
  max_non_zero_image = adapted_image
  for rotation_count in [0]:#[0, 3, 1, 2]:
    rotated_image = np.rot90(adapted_image, k=rotation_count, axes=(1,2))
    for pos_neg in [False, True]:
      if pos_neg:
        rotated_image = 1 - rotated_image
      mask = model[0].predict(rotated_image)
      # Go back to 512x512 image format
      mask = np.squeeze(np.squeeze(mask, axis=0), axis=2)
      # Use the 0-255 pixel value range
      mask = denormalizeImage(mask)
      # Erode the mask
      mask = cv2.erode(mask, np.ones((3,3), np.uint8), iterations=3)

      non_zero_pixels = np.count_nonzero(mask)
      if non_zero_pixels > max_non_zero_pixels:
        max_non_zero_pixels = non_zero_pixels
        max_non_zero_image = rotated_image
  return max_non_zero_image

In [ ]:
image_files = df_test_bad_images.ImageFile.to_numpy()

In [ ]:
BATCH_SIZE = 9

for stride in range(int(len(image_files)/BATCH_SIZE)):
  initial_images = []
  segmented_images = []
  image_file_batch = image_files[stride*BATCH_SIZE : (stride+1)*BATCH_SIZE].tolist()
  for image_file in image_file_batch:
    image = cv2.imread(BASE_PATH+"TestSetPreprocessed/" + image_file, cv2.IMREAD_GRAYSCALE)
    initial_images.append(image)
    mask = model[0].predict(get_max_non_zero_image(image))
    segmented_images.append(np.uint8(np.squeeze(np.squeeze(mask, axis=0), axis=2)*255))
  printExampleImages(initial_images, image_file_batch)
  printExampleImages(segmented_images, image_file_batch)

## Add false negative images

In [ ]:
additional_images_to_be_saved = [
                                 ["P_3_178.png", True],
                                 ["P_3_3.png", False],
                                 ["P_3_78.png", True],
                                 ["P_3_58.png", True],
                                 ["P_3_390.png", True],
]

In [ ]:
corrected_images = []

for image_file, to_be_inverted in additional_images_to_be_saved:
  image = cv2.imread(BASE_PATH+"TestSetPreprocessed/"+image_file, cv2.IMREAD_GRAYSCALE)
  showImage(image, image_file)
  if to_be_inverted:
    image = 255 - image
  image = np.expand_dims(np.expand_dims(image / 255, axis = 0), axis=3)
  corrected_images.append(image)
  image = np.squeeze(np.squeeze(np.uint8(model[0].predict(image) * 255), axis = 0), axis = 2)
  showImage(image, image_file)

In [ ]:
bs_score = []

for index, image in enumerate(corrected_images):
  print("\rProcessing: "+str(index+1)+"/"+str(len(corrected_images)), end="")
  bs_score.append(np.squeeze(model[-1].predict(image), axis = 0))
bs_score = np.array(bs_score)

In [ ]:
argmax = np.argmax(bs_score, axis = 3)
argmax = np.reshape(argmax, (len(corrected_images), 6))
probabilities = np.max(bs_score, axis = 3)
probabilities = np.reshape(probabilities, (len(corrected_images), 6))
confidence = np.mean(probabilities, axis=1)

In [ ]:
for i in range(len(corrected_images)):
  index = df_test[df_test.ImageFile == additional_images_to_be_saved[i][0]].index[0]
  print("Index: ", index)
  for j in range(6):
    df_test.at[index, "Brixia_score_"+str(j)] = argmax[i,j]
  for j in range(6):
    df_test.at[index, "Brixia_score_prob_"+str(j)] = probabilities[i,j]
  df_test.at[index, "Brixia_score_correctness"] = 1
  df_test.at[index, "Brixia_score_confidence"] = confidence[i]

In [ ]:
df_test.to_excel(BASE_PATH + "Candidate_TestSet_brixiascore.xls")

# Can we detect if an image is inverted from its histogram?

In [ ]:
PATH = BASE_PATH + "TrainSetAdjusted/"

file_names = os.listdir(PATH)

file_names = file_names[:4]

plt.figure(figsize=(12, 8))
for i, file_name in enumerate(file_names):
  image = cv2.imread(PATH + file_name, cv2.IMREAD_GRAYSCALE)
  ax = plt.subplot(4, 4, i*4+1)
  plt.imshow(image, cmap = 'gray', interpolation = 'bicubic', vmin=0, vmax=255)
  ax.set_title(file_name)
  plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis

  ax = plt.subplot(4, 4, i*4+3)
  plt.hist(image.ravel(),256,[0,256]);
  ax.set_title(file_name)
  plt.xticks([])

  image = 255 - image
  ax = plt.subplot(4, 4, i*4+2)
  plt.imshow(image, cmap = 'gray', interpolation = 'bicubic', vmin=0, vmax=255)
  ax.set_title(file_name + " inverted")
  plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis

  ax = plt.subplot(4, 4, i*4+4)
  plt.hist(image.ravel(),256,[0,256]);
  ax.set_title(file_name + " inverted")
  plt.xticks([])


# Explainability maps

In [ ]:
TF_BATCH_SIZE = 1

def get_explainability_map(image, idx=None):

    logger.info("Explainability map extraction")
    logger.debug(f"Image shape: {image.shape}")
    bsp = model[-1].predict(image)[0]
    seg = model[0].predict(image)[0]

    logger.debug("Get SLIC")
    segments = slic(Image.fromarray(255 * image[0, :, :, 0]).convert('RGB'),
                        n_segments=180, compactness=10, sigma=1)

    segments = (seg[:, :, 0] > .5) * segments #tweak
    logger.debug(f"SLIC done, found {len(np.unique(segments)[1:])} segments")

    logger.debug("Get predictions for explainability")
    mimages = []

    requests = []
    for i, z in enumerate(np.unique(segments)[1:]):
        mimage = image[0, :, :, 0] * (1 - (segments == z))
        mimages.append(mimage)

    resp = model[-1].predict(np.expand_dims(mimages, axis=-1))

    assert len(resp) > 10, logger.error('The segmentation did not found anything. Probably a wrong image?')

    p = np.array(resp)
    logger.debug(f"Predictions done. len(predictions): {len(resp)}")

    col_dict_class = [(15, 250, 20), (255, 215, 0), (149, 31, 22), (38, 38, 38)]

    w = (bsp - p).max(axis=(1, 2))  # take the max (future: consider also the mean)
    w = np.clip(w, 0, 1)  # take the positive only (the superpixel is respondible for the class)
    w /= w.max()
    wneg = (p - bsp).max(axis=(1, 2))  # take the max (future: consider also the mean)
    wneg = np.clip(wneg, 0, 1)
    wneg /= wneg.max()
    dispersion = np.std(bsp - p, axis=-1)

    pr = np.zeros((512, 512, 4))
    cmaps = np.ones((512, 512, 3))
    cmaps += np.expand_dims(segments == 0, axis=-1) * np.array([245, 245, 245])
    cmaps_neg = cmaps.copy()
    dispersion_map = np.zeros((512, 512, 1))
    cmaps_map = np.zeros((512, 512, 1))
    pmaps = np.zeros((512, 512, 1))

    for i, z in enumerate(np.unique(segments)[1:]):
        # if np.sum(segments==z)>256:
        pr += np.expand_dims(segments == z, axis=-1) * w[i]  # np.sum(p[-1], axis=(0,1,2))

        cmaps += np.expand_dims(segments == z, axis=-1) * np.array(col_dict_class[np.argmax(w[i])])
        cmaps_neg += np.expand_dims(segments == z, axis=-1) * np.array(col_dict_class[np.argmax(wneg[i])])
        cmaps_map += np.expand_dims(segments == z, axis=-1) * np.argmax(w[i])
        pmaps += np.expand_dims(segments == z, axis=-1) * w[i].max()

        dispersion_map += np.expand_dims(segments == z, axis=-1) * dispersion[i].max()

    cmaps[cmaps > 1] = cmaps[cmaps > 1] - 1
    cmaps_neg[cmaps_neg > 1] = cmaps_neg[cmaps_neg > 1] - 1

    contours = measure.find_contours(seg[:, :, 0], 0.5)
    mx = dispersion_map.max()
    for contour in contours:
        rr, cc = polygon_perimeter(contour[:, 0], contour[:, 1], cmaps.shape)
        cmaps[rr, cc] = (0, 0, 100)
        dispersion_map[rr, cc] = mx

    expl_map = np.dstack(
        (cmaps.astype('uint8'), (255 * dispersion_map / dispersion_map.max()).astype('uint8')[:, :, 0]))
    logger.debug("[EME] completed.")

    return Image.fromarray(expl_map)

In [ ]:
image = cv2.imread(BASE_PATH+"TrainSetAdjusted/P_1_60.png", cv2.IMREAD_GRAYSCALE)
image = np.expand_dims(np.expand_dims(image / 255, axis=0), axis=-1)

In [ ]:
image.shape

In [ ]:
get_explainability_map(image)

In [ ]:
bs_score = np.squeeze(model[-1].predict(image), axis = 0)
argmax = np.argmax(bs_score, axis = 2)

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(argmax, cmap = 'Reds')
# Loop over data dimensions and create text annotations.
for i in range(argmax.shape[0]):
    for j in range(argmax.shape[1]):
        color = "k" if argmax[i, j] <=1 else "w"
        ax.text(j, i, argmax[i, j], ha="center", va="center", color=color, size="xx-large")
plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
plt.show()

# EXTRA: improve algorithm (not done)

Here the initial goal is to tune the algorithm in order to understand why it discard some good image. This is partially developed. The motivations behind each rejection are all different. It take a lot of time for just a little gain (21 false positive images out of 1600 --> 1.3% of the images)

In [ ]:
image = cv2.imread(BASE_PATH+"BadImages/P_626.png", cv2.IMREAD_GRAYSCALE)
#adapted_image = 1 - np.expand_dims(np.expand_dims(image / 255, axis=2), axis=0)
adapted_image = np.expand_dims(np.expand_dims(image / 255, axis=2), axis=0)
mask = model[0].predict(adapted_image)
mask = np.squeeze(np.squeeze(mask, axis=0), axis=2)
# Use the 0-255 pixel value range
mask = denormalizeImage(mask)
# Erode the mask
mask = cv2.erode(mask, np.ones((3,3), np.uint8), iterations=3)
# Find connected components
num_regions, labels, stats, _ = cv2.connectedComponentsWithStats(mask, 8, cv2.CV_32S)

In [ ]:
showImage(mask, "Mask")

In [ ]:
canny_output = cv2.Canny(mask, 100, 200)
contours, _ = cv2.findContours(canny_output, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# Keep just the 2 longest contours
argsort_contour = np.argsort([len(val) for val in contours])[::-1]#[-2:]
# Redefine contours with just the 2 longest contours
contours = [contours[i] for i in argsort_contour]
# Find the ellipses for each contour
minEllipses = [None]*len(contours)
for i, c in enumerate(contours):
  if c.shape[0] > 5:
    minEllipses[i] = cv2.fitEllipse(c)

# Draw contours + rotated rects + ellipses
    
drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)
    
for i, c in enumerate(contours):
  color = 255, 255, 255
  # contour
  cv2.drawContours(drawing, contours, i, color)
  # ellipse
  if c.shape[0] > 5:
    cv2.ellipse(drawing, minEllipses[i], color, 2)
showImage(drawing, "Fitted ellipse")

In [ ]:
contours

In [ ]:
canny_output = cv2.Canny(mask, 100, 200)
contours, _ = cv2.findContours(canny_output, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

argsort_contour = np.argsort([len(val) for val in contours])[::-1]
contours = [contours[i] for i in argsort_contour]

if len(contours) > 3:
  contours = contours[:3]

minEllipses = [None]*len(contours)
for i, c in enumerate(contours):
  if c.shape[0] > 5:
    minEllipses[i] = cv2.fitEllipse(c)
  
ellipses = [None]*2
ellipses[0] = minEllipses[0]

if len(contours) > 2 and np.abs(len(contours[2])-len(contours[1])) < 50 :
  if np.tan(np.deg2rad(90 - minEllipses[0][2])) * np.tan(np.deg2rad(90 - minEllipses[1][2])) < 0:
    ellipses[1] = minEllipses[1]
  else:
    ellipses[1] = minEllipses[2]
else:
  ellipses[1] = minEllipses[1]

# Draw contours + rotated rects + ellipses
    
drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)

color = 255, 255, 255 
for i, c in enumerate(contours):
  # contour
  cv2.drawContours(drawing, contours, i, color)

for i, c in enumerate(ellipses):
  cv2.ellipse(drawing, ellipses[i], color, 2)

showImage(drawing, "New algorithm")